# Deep Nested Pydantic Variants Playground

This notebook demonstrates how to work with deeply nested BaseModel schemas (3 levels deep) and use transformers to create variants with different behaviors. We'll explore:

1. Creating complex nested schemas
2. Applying transformers like Filter, Optional, and Rename
3. Using SwitchNested to switch all nested models to specific variants
4. Building and attaching variant models
5. Testing the resulting variants with real data

## 1. Import Required Libraries and Transformers

Import all the necessary modules from our pydantic_variants system.

In [1]:
from typing import Optional
from datetime import datetime
from pydantic import BaseModel, Field

# Import our core transformers and utilities
from core import (
    VariantContext, 
    VariantPipe,
    Filter, 
    Optional as OptionalTransformer, 
    Rename, 
    Build, 
    Attach,
    SetFields
)
from transformers.switch_variant import SwitchNested
from pydantic.fields import FieldInfo
from deepdiff import DeepDiff
print("All modules imported successfully!")

All modules imported successfully!


## 2. Define Deeply Nested BaseModel Schema (3 Levels)

Let's create a realistic example with 3 levels of nesting:
- **Level 3**: `Address` - The deepest level with basic fields
- **Level 2**: `Contact` - Contains an Address and contact info
- **Level 1**: `User` - The top level containing Contact and user data

In [2]:
# Level 3: Address (deepest level)
class Address(BaseModel):
    """Address information - Level 3 (deepest)"""
    street: str
    city: str
    state: str
    postal_code: str
    country: str 
    apartment: Optional[str] = None
    is_primary: bool = True

class AddressB(BaseModel):
    """Address information - Level 3 (deepest)"""
    street: str
    city: str
    blabla: str
    postal_code: str
    country: str 
    apartment: Optional[str] = None
    is_primary: bool = True

# Level 2: Contact (middle level)
class Contact(BaseModel):
    """Contact information - Level 2 (middle)"""
    email: str
    phone: str
    address: Optional[Address|AddressB]  # Nested Level 3
    emergency_contact_name: str
    emergency_contact_phone: str
    preferred_contact_method: str = "email"

# Level 1: User (top level)
class User(BaseModel):
    """User model - Level 1 (top level)"""
    user_id: int
    username: str
    first_name: str
    last_name: str
    contact: Contact  # Nested Level 2 (which contains Level 3)
    created_at: datetime
    last_login: Optional[datetime] = None
    is_active: bool = True
    role: str = "user"
    metadata: dict = Field(default_factory=dict)

print("Nested models defined:")
print(f"Address fields: {list(Address.model_fields.keys())}")
print(f"Contact fields: {list(Contact.model_fields.keys())}")
print(f"User fields: {list(User.model_fields.keys())}")

Nested models defined:
Address fields: ['street', 'city', 'state', 'postal_code', 'country', 'apartment', 'is_primary']
Contact fields: ['email', 'phone', 'address', 'emergency_contact_name', 'emergency_contact_phone', 'preferred_contact_method']
User fields: ['user_id', 'username', 'first_name', 'last_name', 'contact', 'created_at', 'last_login', 'is_active', 'role', 'metadata']


## 3. Create Variants for Each Level

First, let's create different variants for each level of our nested models. We'll create:
- **Input** variants - for data validation when receiving data
- **Output** variants - for API responses with filtered/transformed fields

In [3]:
# Create Input variant for Address (Level 3) - make some fields optional
address_input_pipeline = VariantPipe().then(VariantContext("Input")
).then(
    OptionalTransformer(include_only=['apartment', 'country', 'is_primary'])
).then(
    Build()
).then(
    Attach()
)


Address_input = address_input_pipeline.process(Address).current_variant
address_input_pipeline.process(AddressB)
DeepDiff(Address, Address_input, exclude_regex_paths=[r".*_original_assignment",r".*_attributes_set"])

{'type_changes': {"root.model_fields['country'].annotation": {'old_type': str,
   'new_type': typing.Optional[str]},
  "root.model_fields['country'].default": {'old_type': pydantic_core._pydantic_core.PydanticUndefinedType,
   'new_type': NoneType,
   'old_value': PydanticUndefined,
   'new_value': None}},
 'attribute_removed': ['root._variants']}

In [4]:
# Create Output variant for Address - hide internal fields, rename some fields  
address_output_pipeline = VariantPipe().then(VariantContext("Output")).then(
    Filter(exclude=['is_primary'])  # Remove internal field
).then(
    Rename(mapping={'postal_code': 'zip', 'apartment': 'apt'})  # API-friendly names
).then(
    Build()
).then(
    Attach()
)

address_output_context = address_output_pipeline.process(Address)
address_output_pipeline.process(AddressB)
Address_output = address_output_context.current_variant
Address_output = address_output_context.current_variant

print("Address Output variant created!")
print(f"Address Output fields: {list(Address._variants['Output'].model_fields.keys())}")

print("\n📊 DeepDiff Analysis:")
diff = DeepDiff(Address, Address_output, exclude_regex_paths=[r".*_original_assignment", r".*_attributes_set"])
if diff:
    print(diff)
else:
    print("No significant differences found")

Address Output variant created!
Address Output fields: ['street', 'city', 'state', 'zip', 'country', 'apt']

📊 DeepDiff Analysis:
{'values_changed': {'root.model_fields': {'new_value': {'street': FieldInfo(annotation=str, required=True), 'city': FieldInfo(annotation=str, required=True), 'blabla': FieldInfo(annotation=str, required=True), 'zip': FieldInfo(annotation=str, required=True), 'country': FieldInfo(annotation=str, required=True), 'apt': FieldInfo(annotation=Union[str, NoneType], required=False, default=None)}, 'old_value': {'street': FieldInfo(annotation=str, required=True), 'city': FieldInfo(annotation=str, required=True), 'state': FieldInfo(annotation=str, required=True), 'postal_code': FieldInfo(annotation=str, required=True), 'country': FieldInfo(annotation=str, required=True), 'apartment': FieldInfo(annotation=Union[str, NoneType], required=False, default=None), 'is_primary': FieldInfo(annotation=bool, required=False, default=True)}}}, 'attribute_removed': ['root._variants

In [5]:
Address_output.model_fields

{'street': FieldInfo(annotation=str, required=True),
 'city': FieldInfo(annotation=str, required=True),
 'blabla': FieldInfo(annotation=str, required=True),
 'zip': FieldInfo(annotation=str, required=True),
 'country': FieldInfo(annotation=str, required=True),
 'apt': FieldInfo(annotation=Union[str, NoneType], required=False, default=None)}

In [6]:
AddressB._variants['Output']

__main__.AddressBOutput

In [7]:
# Create Input variant for Contact (Level 2) 
contact_input_pipeline = VariantPipe().then(VariantContext("Input")).then(
    OptionalTransformer(include_only=['emergency_contact_name', 'emergency_contact_phone', 'preferred_contact_method'])
).then(
    SwitchNested('Input')  # Switch nested Address to Input variant
).then(
    Build()
).then(
    Attach()
)

contact_input_context = contact_input_pipeline.process(Contact)
Contact_input = contact_input_context.current_variant

print("Contact Input variant created!")
print(f"Contact Input uses Address variant: {contact_input_context.current_variant.model_fields['address'].annotation}")
print(f"Contact Input required fields: {[name for name, field in Contact._variants['Input'].model_fields.items() if field.is_required()]}")

print("\n📊 DeepDiff Analysis:")
diff = DeepDiff(Contact, Contact_input, exclude_regex_paths=[r".*_original_assignment", r".*_attributes_set"])
if diff:
    print(diff)
else:
    print("No significant differences found")

Contact Input variant created!
Contact Input uses Address variant: typing.Union[__main__.AddressInput, __main__.AddressBInput, NoneType]
Contact Input required fields: ['email', 'phone', 'address']

📊 DeepDiff Analysis:
{'type_changes': {"root.model_fields['emergency_contact_name'].annotation": {'old_type': <class 'str'>, 'new_type': typing.Optional[str]}, "root.model_fields['emergency_contact_name'].default": {'old_type': <class 'pydantic_core._pydantic_core.PydanticUndefinedType'>, 'new_type': <class 'NoneType'>, 'old_value': PydanticUndefined, 'new_value': None}, "root.model_fields['emergency_contact_phone'].annotation": {'old_type': <class 'str'>, 'new_type': typing.Optional[str]}, "root.model_fields['emergency_contact_phone'].default": {'old_type': <class 'pydantic_core._pydantic_core.PydanticUndefinedType'>, 'new_type': <class 'NoneType'>, 'old_value': PydanticUndefined, 'new_value': None}}, 'attribute_removed': ['root._variants']}


In [8]:
Contact_input.model_fields

{'email': FieldInfo(annotation=str, required=True),
 'phone': FieldInfo(annotation=str, required=True),
 'address': FieldInfo(annotation=Union[AddressInput, AddressBInput, NoneType], required=True),
 'emergency_contact_name': FieldInfo(annotation=Union[str, NoneType], required=False, default=None),
 'emergency_contact_phone': FieldInfo(annotation=Union[str, NoneType], required=False, default=None),
 'preferred_contact_method': FieldInfo(annotation=str, required=False, default='email')}

In [9]:
Contact.model_fields

{'email': FieldInfo(annotation=str, required=True),
 'phone': FieldInfo(annotation=str, required=True),
 'address': FieldInfo(annotation=Union[Address, AddressB, NoneType], required=True),
 'emergency_contact_name': FieldInfo(annotation=str, required=True),
 'emergency_contact_phone': FieldInfo(annotation=str, required=True),
 'preferred_contact_method': FieldInfo(annotation=str, required=False, default='email')}

In [10]:
# Create Output variant for Contact - filter sensitive data
contact_output_pipeline = VariantPipe().then(VariantContext("Output")).then(
    Filter(exclude=['emergency_contact_name', 'emergency_contact_phone'])  # Hide emergency info in public API
).then(
    SwitchNested('Output')  # Switch nested Address to Output variant  
).then(
    Build()
).then(
    Attach()
)

contact_output_context = contact_output_pipeline.process(Contact)
Contact_output = contact_output_context.current_variant

print("Contact Output variant created!")
print(f"Contact Output fields: {list(Contact._variants['Output'].model_fields.keys())}")
print(f"Contact Output uses Address variant: {contact_output_context.current_variant.model_fields['address'].annotation}")

print("\n📊 DeepDiff Analysis:")
diff = DeepDiff(Contact, Contact_output, exclude_regex_paths=[r".*_original_assignment", r".*_attributes_set"])
if diff:
    print(diff)
else:
    print("No significant differences found")

Contact Output variant created!
Contact Output fields: ['email', 'phone', 'address', 'preferred_contact_method']
Contact Output uses Address variant: typing.Union[__main__.AddressOutput, __main__.AddressBOutput, NoneType]

📊 DeepDiff Analysis:
{'dictionary_item_removed': ["root.model_fields['emergency_contact_name']", "root.model_fields['emergency_contact_phone']"], 'attribute_removed': ['root._variants']}


In [11]:
Contact_output.model_fields['address'].annotation.model_fields

AttributeError: model_fields

## 4. Create User Variants with Deep Nesting (Level 1)

Now let's create User variants that will automatically use the appropriate nested variants through the SwitchNested transformer.

In [19]:
# Create Input variant for User (Level 1) - for user registration/creation
user_input_pipeline = VariantPipe().then(VariantContext("Input")).then(
    OptionalTransformer(include_only=['last_login', 'is_active', 'role', 'metadata'])  # Make optional for input
).then(
    SetFields({  # Add a field for input processing
        'created_at': FieldInfo(annotation=datetime, default_factory=datetime.now)
    })
).then(
    SwitchNested('Input')  # This will switch Contact to Input variant, which in turn uses Address Input variant
).then(
    Build()
).then(
    Attach()
)

user_input_context = user_input_pipeline.process(User)
User_input = user_input_context.current_variant

print("User Input variant created!")
print(f"User Input required fields: {[name for name, field in User._variants['Input'].model_fields.items() if field.is_required()]}")

# Let's inspect the nested structure
user_input_model = User._variants['Input']
contact_annotation = user_input_model.model_fields['contact'].annotation
print(f"User Input -> Contact variant: {contact_annotation}")

# Check if the Contact variant has the Address variant
if hasattr(contact_annotation, 'model_fields'):
    address_annotation = contact_annotation.model_fields['address'].annotation
    print(f"User Input -> Contact -> Address variant: {address_annotation}")

print("\n📊 DeepDiff Analysis:")
diff = DeepDiff(User, User_input, exclude_regex_paths=[r".*_original_assignment", r".*_attributes_set"])
if diff:
    print(diff)
else:
    print("No significant differences found")

User Input variant created!
User Input required fields: ['user_id', 'username', 'first_name', 'last_name', 'contact']
User Input -> Contact variant: <class '__main__.ContactInput'>
User Input -> Contact -> Address variant: typing.Union[__main__.AddressInput, __main__.AddressBInput, NoneType]

📊 DeepDiff Analysis:
{'type_changes': {"root.model_fields['contact'].annotation.model_fields['emergency_contact_name'].annotation": {'old_type': <class 'str'>, 'new_type': typing.Optional[str]}, "root.model_fields['contact'].annotation.model_fields['emergency_contact_name'].default": {'old_type': <class 'pydantic_core._pydantic_core.PydanticUndefinedType'>, 'new_type': <class 'NoneType'>, 'old_value': PydanticUndefined, 'new_value': None}, "root.model_fields['contact'].annotation.model_fields['emergency_contact_phone'].annotation": {'old_type': <class 'str'>, 'new_type': typing.Optional[str]}, "root.model_fields['contact'].annotation.model_fields['emergency_contact_phone'].default": {'old_type': <

In [ ]:
diff

{'type_changes': {"root.model_fields['contact'].annotation.model_fields['emergency_contact_name'].annotation": {'old_type': str,
   'new_type': typing.Optional[str]},
  "root.model_fields['contact'].annotation.model_fields['emergency_contact_name'].default": {'old_type': pydantic_core._pydantic_core.PydanticUndefinedType,
   'new_type': NoneType,
   'old_value': PydanticUndefined,
   'new_value': None},
  "root.model_fields['contact'].annotation.model_fields['emergency_contact_phone'].annotation": {'old_type': str,
   'new_type': typing.Optional[str]},
  "root.model_fields['contact'].annotation.model_fields['emergency_contact_phone'].default": {'old_type': pydantic_core._pydantic_core.PydanticUndefinedType,
   'new_type': NoneType,
   'old_value': PydanticUndefined,
   'new_value': None},
  "root.model_fields['created_at'].default_factory": {'old_type': NoneType,
   'new_type': builtin_function_or_method,
   'old_value': None,
   'new_value': <function datetime.now(tz=None)>}},
 'attri

In [15]:
# Create Output variant for User - for API responses  
user_output_pipeline = VariantPipe().then(VariantContext("Output")).then(
    Filter(exclude=['metadata'])  # Hide internal metadata in public API
).then(
    Rename(mapping={'user_id': 'id'})  # API-friendly field name
).then(
    SwitchNested('Output')  # This cascades down: Contact -> Output, Address -> Output
).then(
    Build()
).then(
    Attach()
)

user_output_context = user_output_pipeline.process(User)
User_output = user_output_context.current_variant

print("User Output variant created!")
print(f"User Output fields: {list(User._variants['Output'].model_fields.keys())}")

# Let's inspect the full nested structure
user_output_model = User._variants['Output']
contact_annotation = user_output_model.model_fields['contact'].annotation
print(f"User Output -> Contact variant: {contact_annotation}")

# Check the deep nesting
if hasattr(contact_annotation, 'model_fields'):
    address_annotation = contact_annotation.model_fields['address'].annotation
    print(f"User Output -> Contact -> Address variant: {address_annotation}")

print("\n📊 DeepDiff Analysis:")
diff = DeepDiff(User, User_output, exclude_regex_paths=[r".*_original_assignment", r".*_attributes_set"])
if diff:
    print(diff)
else:
    print("No significant differences found")

User Output variant created!
User Output fields: ['id', 'username', 'first_name', 'last_name', 'contact', 'created_at', 'last_login', 'is_active', 'role']
User Output -> Contact variant: <class '__main__.ContactOutput'>
User Output -> Contact -> Address variant: typing.Union[__main__.AddressOutput, __main__.AddressBOutput, NoneType]

📊 DeepDiff Analysis:
{'dictionary_item_added': ["root.model_fields['id']"], 'dictionary_item_removed': ["root.model_fields['user_id']", "root.model_fields['metadata']", "root.model_fields['contact'].annotation.model_fields['emergency_contact_name']", "root.model_fields['contact'].annotation.model_fields['emergency_contact_phone']"], 'attribute_removed': ['root._variants', "root.model_fields['contact'].annotation._variants"]}


In [16]:
address_annotation

typing.Union[__main__.AddressOutput, __main__.AddressBOutput, NoneType]

In [ ]:
User_output.model_json_schema()

{'$defs': {'AddressOutput': {'description': 'Address information - Level 3 (deepest)',
   'properties': {'street': {'title': 'Street', 'type': 'string'},
    'city': {'title': 'City', 'type': 'string'},
    'state': {'title': 'State', 'type': 'string'},
    'zip': {'title': 'Zip', 'type': 'string'},
    'country': {'title': 'Country', 'type': 'string'},
    'apt': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None,
     'title': 'Apt'}},
   'required': ['street', 'city', 'state', 'zip', 'country'],
   'title': 'AddressOutput',
   'type': 'object'},
  'ContactOutput': {'description': 'Contact information - Level 2 (middle)',
   'properties': {'email': {'title': 'Email', 'type': 'string'},
    'phone': {'title': 'Phone', 'type': 'string'},
    'address': {'$ref': '#/$defs/AddressOutput'},
    'preferred_contact_method': {'default': 'email',
     'title': 'Preferred Contact Method',
     'type': 'string'}},
   'required': ['email', 'phone', 'address'],
   'title': 'Con

## 5. Test the Variants with Real Data

Let's create some test data and see how our variants behave differently:

In [21]:
# Test data for Input variant (simulating user registration)
input_data = {
    "user_id": 123,
    "username": "johndoe",
    "first_name": "John", 
    "last_name": "Doe",
    "contact": {
        "email": "john@example.com",
        "phone": "+1-555-0123",
        "address": {
            "street": "123 Main St",
            "city": "Anytown", 
            "state": "CA",
            "postal_code": "12345"
            # Note: apartment, country, is_primary are optional in Input variant
        }
        # Note: emergency contacts are optional in Input variant
    }
    # Note: created_at will be auto-generated, other fields have defaults
}

# Create instance using Input variant
UserInput = User._variants['Input']
user_input_instance = UserInput(**input_data)

print("✅ User Input variant instance created successfully!")
print(f"Created at: {user_input_instance.created_at}")
print(f"User role: {user_input_instance.role}")
print(f"Contact address country: {user_input_instance.contact.address.country}")
print(f"Is primary address: {user_input_instance.contact.address.is_primary}")

# Show the full nested structure
print("\n📝 Input instance structure:")
print(f"User ID: {user_input_instance.user_id}")
print(f"Contact email: {user_input_instance.contact.email}")
print(f"Address: {user_input_instance.contact.address.street}, {user_input_instance.contact.address.city}")

✅ User Input variant instance created successfully!
Created at: 2025-08-03 00:53:02.058877
User role: user
Contact address country: None
Is primary address: True

📝 Input instance structure:
User ID: 123
Contact email: john@example.com
Address: 123 Main St, Anytown


In [20]:
User._variants

{'Output': __main__.UserOutput, 'Input': __main__.UserInput}

In [22]:
# Now let's create a full user instance with all data and then convert to Output variant
full_user_data = {
    "user_id": 123,
    "username": "johndoe", 
    "first_name": "John",
    "last_name": "Doe",
    "contact": {
        "email": "john@example.com",
        "phone": "+1-555-0123", 
        "address": {
            "street": "123 Main St",
            "city": "Anytown",
            "state": "CA", 
            "postal_code": "12345",
            "country": "USA",
            "apartment": "Apt 4B",
            "is_primary": True
        },
        "emergency_contact_name": "Jane Doe",
        "emergency_contact_phone": "+1-555-0456", 
        "preferred_contact_method": "email"
    },
    "created_at": datetime.now(),
    "last_login": datetime.now(),
    "is_active": True,
    "role": "admin",
    "metadata": {"source": "manual_registration", "ip": "192.168.1.1"}
}

# Create original user instance
original_user = User(**full_user_data)
print("✅ Original User instance created")

# Convert to Output variant for API response
UserOutput = User._variants['Output']

# Create the output data by extracting compatible fields
output_data = original_user.model_dump()

# Transform data for Output variant (field renames, exclusions are handled by model definition)
output_data['id'] = output_data.pop('user_id')  # user_id -> id rename
output_data.pop('metadata', None)  # Remove metadata  

# Transform nested contact data
contact_data = output_data['contact']
contact_data.pop('emergency_contact_name', None)  # Filtered out in Output variant
contact_data.pop('emergency_contact_phone', None)

# Transform nested address data  
address_data = contact_data['address']
address_data['zip'] = address_data.pop('postal_code')  # postal_code -> zip rename
address_data['apt'] = address_data.pop('apartment')   # apartment -> apt rename
address_data.pop('is_primary', None)  # Filtered out in Output variant

user_output_instance = UserOutput(**output_data)

print("✅ User Output variant instance created!")
print("\n📤 Output instance (API response format):")
print(f"ID: {user_output_instance.id}")  # Renamed field
print(f"Contact email: {user_output_instance.contact.email}")
print(f"Address: {user_output_instance.contact.address.street}, {user_output_instance.contact.address.city}")
print(f"ZIP: {user_output_instance.contact.address.zip}")  # Renamed field
print(f"Apt: {user_output_instance.contact.address.apt}")   # Renamed field

# Show what fields were filtered out
print("\n🔒 Fields filtered out from Output:")
print("- User metadata: Not present")
print("- Emergency contacts: Not present") 
print("- Address is_primary: Not present")

✅ Original User instance created
✅ User Output variant instance created!

📤 Output instance (API response format):
ID: 123
Contact email: john@example.com
Address: 123 Main St, Anytown
ZIP: 12345
Apt: Apt 4B

🔒 Fields filtered out from Output:
- User metadata: Not present
- Emergency contacts: Not present
- Address is_primary: Not present


## 6. Compare Schema Differences

Let's examine how the variants differ from the original models at each level:

In [23]:
def compare_schemas(original_model, variant_model, model_name):
    """Compare original and variant model schemas"""
    print(f"\n📊 {model_name} Schema Comparison:")
    print("=" * 50)
    
    orig_fields = set(original_model.model_fields.keys())
    var_fields = set(variant_model.model_fields.keys())
    
    print(f"Original fields: {sorted(orig_fields)}")
    print(f"Variant fields:  {sorted(var_fields)}")
    
    # Show added/removed fields
    added = var_fields - orig_fields
    removed = orig_fields - var_fields
    
    if added:
        print(f"➕ Added fields: {sorted(added)}")
    if removed:
        print(f"➖ Removed fields: {sorted(removed)}")
    
    # Show required vs optional differences
    orig_required = {name for name, field in original_model.model_fields.items() if field.is_required()}
    var_required = {name for name, field in variant_model.model_fields.items() if field.is_required()}
    
    newly_optional = orig_required - var_required
    if newly_optional:
        print(f"🔓 Made optional: {sorted(newly_optional)}")

# Compare all three levels
print("🔍 DEEP NESTED SCHEMA ANALYSIS")
print("=" * 60)

# Level 3: Address
compare_schemas(Address, Address._variants['Input'], "Address Input")
compare_schemas(Address, Address._variants['Output'], "Address Output")

# Level 2: Contact  
compare_schemas(Contact, Contact._variants['Input'], "Contact Input")
compare_schemas(Contact, Contact._variants['Output'], "Contact Output")

# Level 1: User
compare_schemas(User, User._variants['Input'], "User Input") 
compare_schemas(User, User._variants['Output'], "User Output")

🔍 DEEP NESTED SCHEMA ANALYSIS

📊 Address Input Schema Comparison:
Original fields: ['apartment', 'city', 'country', 'is_primary', 'postal_code', 'state', 'street']
Variant fields:  ['apartment', 'city', 'country', 'is_primary', 'postal_code', 'state', 'street']
🔓 Made optional: ['country']

📊 Address Output Schema Comparison:
Original fields: ['apartment', 'city', 'country', 'is_primary', 'postal_code', 'state', 'street']
Variant fields:  ['apt', 'city', 'country', 'state', 'street', 'zip']
➕ Added fields: ['apt', 'zip']
➖ Removed fields: ['apartment', 'is_primary', 'postal_code']
🔓 Made optional: ['postal_code']

📊 Contact Input Schema Comparison:
Original fields: ['address', 'email', 'emergency_contact_name', 'emergency_contact_phone', 'phone', 'preferred_contact_method']
Variant fields:  ['address', 'email', 'emergency_contact_name', 'emergency_contact_phone', 'phone', 'preferred_contact_method']
🔓 Made optional: ['emergency_contact_name', 'emergency_contact_phone']

📊 Contact Outpu

## 7. Summary and Key Insights

This playground demonstrated several powerful features of the pydantic_variants system:

### 🔑 Key Features Demonstrated

1. **Deep Nesting Support**: Created 3-level nested models (`User` → `Contact` → `Address`)

2. **Cascading Variant Switching**: The `SwitchNested` transformer automatically propagated variant selections through all nesting levels

3. **Transformer Composition**: Combined multiple transformers (`Filter`, `Optional`, `Rename`, `SwitchNested`, `Build`, `Attach`) in pipelines

4. **Field-Level Control**: Different transformations at each level:
   - **Address**: Made fields optional, renamed for API compatibility
   - **Contact**: Filtered sensitive emergency contact data 
   - **User**: Added auto-generated fields, renamed IDs

### 🎯 Use Cases

- **Input Variants**: Perfect for user registration/data ingestion with optional fields and sensible defaults
- **Output Variants**: Ideal for API responses with filtered sensitive data and API-friendly field names
- **Nested Consistency**: Ensures all levels use appropriate variants automatically

### 🔧 Architecture Benefits

- **Type Safety**: All variants maintain proper Pydantic validation
- **Reusability**: Transformers can be composed and reused across different models
- **Maintainability**: Changes to transformation logic are centralized in transformer definitions